# 2.3 - Comprobación localización

Mi manera de comprabar esto es dejar que sea el modelo el me diga, con las métricas de error que forma de transformar el dato es mejor, si tener la tupla latitud-longitud, la transformación esférica que realizé anteriormente con las variables x-y-z o realizar un one-hot encoding directamente con los barrios o distritos.  

In [1]:
# librerias

import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
cols=['neighbourhood_cleansed','neighbourhood_group_cleansed',
      'accommodates', 'air_conditioning', 'availability_30', 'availability_365', 'availability_60', 'availability_90',
      'bathrooms', 'bedrooms', 'beds', 'calculated_host_listings_count', 'calculated_host_listings_count_entire_homes',
      'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms', 'cleaning_fee',
      'dishwasher', 'extra_people', 'guests_included','latitude', 'longitude', 'maximum_nights', 'minimum_nights', 
      'number_of_reviews', 'number_of_reviews_ltm', 'room_type_private_room', 'room_type_shared_room', 'security_deposit',
      'x', 'y', 'z', 'price']

In [3]:
# carga de datos
listings=pd.read_csv('../data/transform_data/listings_normal.csv', usecols=cols)

listings=listings[(listings.price>=10) & (listings.price<=196)]  # eliminacion de outliers

# cambio en el tamaño del tipo de dato
for c in listings.select_dtypes(include='int'):
    listings[c]=pd.to_numeric(listings[c], downcast='integer')

for c in listings.select_dtypes(include='float'):
    listings[c]=pd.to_numeric(listings[c], downcast='float')
    
listings.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18936 entries, 0 to 21311
Data columns (total 32 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   neighbourhood_cleansed                        18936 non-null  object 
 1   neighbourhood_group_cleansed                  18936 non-null  object 
 2   latitude                                      18936 non-null  float32
 3   longitude                                     18936 non-null  float32
 4   accommodates                                  18936 non-null  float32
 5   bathrooms                                     18936 non-null  float32
 6   bedrooms                                      18936 non-null  float32
 7   beds                                          18936 non-null  float32
 8   price                                         18936 non-null  int16  
 9   security_deposit                              18936 non-null 

In [4]:
listings.head()

,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,accommodates,bathrooms,bedrooms,beds,price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,x,y,z,room_type_private_room,room_type_shared_room,air_conditioning,dishwasher
0,Hispanoamérica,Chamartín,40.456280,-3.67763,-0.645354,-0.347512,-0.411921,-1.292693,70,-0.459455,-0.556685,0.163400,0.397414,-0.157275,-0.007999,0.832845,1.036105,1.062387,-0.593153,0.531084,-0.041333,-0.326920,-0.301845,-0.131436,-0.103323,0.796991,-0.473457,0.375026,1,0,1,0
1,Cármenes,Latina,40.403412,-3.74084,-1.139633,-0.347512,-0.411921,-0.641980,17,-0.459455,-0.703313,-0.548597,-0.031884,-0.018390,-0.008424,-1.115888,-1.231071,-1.303542,-0.007074,-0.077329,-0.678573,-0.295958,-0.301845,0.094811,-0.103323,0.748058,-0.510948,0.423488,1,0,1,0
2,Legazpi,Arganzuela,40.386951,-3.69304,1.331759,1.052522,1.914222,1.960871,50,0.861337,0.176453,-0.548597,0.090772,0.490853,-0.007522,-0.938731,-1.143872,-1.245836,0.366551,-0.579270,-0.678573,-0.172109,-0.112914,-0.357682,-0.103323,0.765575,-0.470904,0.438342,0,0,1,1
3,Universidad,Centro,40.422020,-3.70395,-0.151076,-0.347512,-1.574992,-0.641980,80,0.421073,0.176453,0.163400,-0.522511,0.027905,-0.007522,1.275739,1.254103,1.206651,1.458122,-0.442377,-0.359953,-0.326920,-0.270356,-0.357682,-0.103323,0.772927,-0.487128,0.406559,0,0,1,0
4,Justicia,Centro,40.419949,-3.69764,0.343203,-0.347512,0.751150,0.659445,115,0.421073,-0.703313,1.587393,-0.522511,-0.064685,-0.008463,1.010003,1.123304,0.918123,1.311603,0.439822,0.004184,-0.326920,-0.270356,-0.357682,-0.103323,0.775268,-0.481796,0.408451,0,0,1,0


**Preparación de los 4 paquete de datos**

In [5]:
df_xyz=listings.drop(columns=['latitude', 'longitude', 'neighbourhood_cleansed', 'neighbourhood_group_cleansed'])

df_lat_lon=listings.drop(columns=['x', 'y', 'z', 'neighbourhood_cleansed', 'neighbourhood_group_cleansed'])

df_barrio=listings.drop(columns=['x', 'y', 'z', 'latitude', 'longitude', 'neighbourhood_group_cleansed'])
df_barrio=pd.get_dummies(df_barrio, prefix='', prefix_sep='')

df_distrito=listings.drop(columns=['x', 'y', 'z', 'latitude', 'longitude', 'neighbourhood_cleansed'])
df_distrito=pd.get_dummies(df_distrito, prefix='', prefix_sep='')

In [6]:
df_xyz.shape

(18936, 28)

In [7]:
df_lat_lon.shape

(18936, 27)

In [8]:
df_barrio.shape

(18936, 152)

In [9]:
df_distrito.shape

(18936, 46)

In [10]:
data={'esféricas': df_xyz, 'latitud-longitud': df_lat_lon, 
      'barrio': df_barrio, 'distrito': df_distrito}

### Testeo de varios modelos en varios paquetes de datos

In [11]:
from sklearn.ensemble import RandomForestRegressor as RFR

from xgboost import XGBRegressor as XGBR

from lightgbm import LGBMRegressor as LGBMR

from catboost import CatBoostRegressor as CTR

from sklearn.model_selection import train_test_split as tts 

from sklearn.metrics import mean_squared_error as mse 
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import r2_score as r2

from tqdm.notebook import tqdm

In [12]:
def test_model_data(modelo: object, data: pd.DataFrame, dataname: str) -> str:
    
    X=data.drop('price', axis=1)
    y=data.price

    X_train, X_test, y_train, y_test = tts(X, y, train_size=0.8, test_size=0.2, random_state=42)
    
    modelo.fit(X_train, y_train)
    
    y_pred=modelo.predict(X_test)
    
    print(dataname)
    print(f'RMSE: {mse(y_test, y_pred, squared=False)}')
    print(f'MAE: {mae(y_test, y_pred)}')
    print(f'R2: {r2(y_test, y_pred)}')
    print()
    return 'Done!'

In [13]:
%%time

print('\n\033[1m' + 'Prueba Random Forest Regressor' + '\033[0m')

[test_model_data(RFR(), d, n) for n,d in tqdm(data.items())]


Prueba Random Forest Regressor


  0%|          | 0/4 [00:00<?, ?it/s]

esféricas
RMSE: 23.198186228052037
MAE: 15.717410242872228
R2: 0.6705536512010941

latitud-longitud
RMSE: 23.059398039315468
MAE: 15.68193505807814
R2: 0.6744838280617724

barrio
RMSE: 23.525056264017877
MAE: 15.71948916415627
R2: 0.6612042300617703

distrito
RMSE: 23.37222262675112
MAE: 15.822342731809139
R2: 0.6655919937641224

CPU times: user 37.7 s, sys: 246 ms, total: 38 s
Wall time: 39.4 s


['Done!', 'Done!', 'Done!', 'Done!']

In [14]:
%%time

print('\n\033[1m' + 'Prueba XG Boosting Regressor' + '\033[0m')

[test_model_data(XGBR(), d, n) for n,d in tqdm(data.items())]


Prueba XG Boosting Regressor


  0%|          | 0/4 [00:00<?, ?it/s]

esféricas
RMSE: 22.60748291015625
MAE: 15.415035247802734
R2: 0.6871176567546005

latitud-longitud
RMSE: 22.395402908325195
MAE: 15.370203018188477
R2: 0.6929604138122123

barrio
RMSE: 22.945114135742188
MAE: 15.653074264526367
R2: 0.6777023962519406

distrito
RMSE: 22.652978897094727
MAE: 15.491287231445312
R2: 0.6858570513410438

CPU times: user 30.3 s, sys: 5.99 s, total: 36.3 s
Wall time: 11.6 s


['Done!', 'Done!', 'Done!', 'Done!']

In [15]:
%%time

print('\n\033[1m' + 'Prueba LightGBM Regressor' + '\033[0m')

[test_model_data(LGBMR(), d, n) for n,d in tqdm(data.items())]


Prueba LightGBM Regressor


  0%|          | 0/4 [00:00<?, ?it/s]

esféricas
RMSE: 22.31358300843586
MAE: 15.282772968407693
R2: 0.6951997889875593

latitud-longitud
RMSE: 22.321120221253018
MAE: 15.304918853648152
R2: 0.6949938398075091

barrio
RMSE: 22.73635614020502
MAE: 15.659719321941182
R2: 0.6835403346621899

distrito
RMSE: 22.761233246210807
MAE: 15.630341002904007
R2: 0.6828474437242844

CPU times: user 2.14 s, sys: 2.63 s, total: 4.76 s
Wall time: 3.64 s


['Done!', 'Done!', 'Done!', 'Done!']

In [16]:
%%time

print('\n\033[1m' + 'Prueba Catboost Regressor' + '\033[0m')

[test_model_data(CTR(verbose=0), d, n) for n,d in tqdm(data.items())]


Prueba Catboost Regressor


  0%|          | 0/4 [00:00<?, ?it/s]

esféricas
RMSE: 21.768811996239744
MAE: 14.882441666578858
R2: 0.7099010913476712

latitud-longitud
RMSE: 21.713858907632996
MAE: 14.82603810439655
R2: 0.7113638913656304

barrio
RMSE: 22.356831249708588
MAE: 15.317336812557775
R2: 0.6940171149343588

distrito
RMSE: 22.167798256429837
MAE: 15.178265646364341
R2: 0.6991695741565194

CPU times: user 24.6 s, sys: 10.2 s, total: 34.7 s
Wall time: 17.3 s


['Done!', 'Done!', 'Done!', 'Done!']

In [17]:
print('\n\033[1m' + 'Catboost Regressor' + '\033[0m')

test_model_data(CTR(verbose=0), df_lat_lon, 'latitud-longitud')


Catboost Regressor
latitud-longitud
RMSE: 21.713858907632996
MAE: 14.82603810439655
R2: 0.7113638913656304



'Done!'

Este último modelo es funciona mejor en términos de MAE que usando la x-y-z, y mejor que usando las variables dummies. Esta será la selección final de variables y también el modelo que vamos a usar. En el próximo notebook entrenaré el modelo, ajustaré hiperparámetros y evaluaré.